In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import urbanpy as up
import contextily as cx
import seaborn as sns
import shapely
import geoplot as gplt
import geoplot.crs as gcrs
from shapely.validation import make_valid
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
sns.set_theme(style="whitegrid")

In [ ]:
data = pd.read_csv("inputs/oferta_alimentaria_14_06_2022.csv")

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
puntos = data[data["Tipo de localización"] == "Punto"]

In [ ]:
puntos_gdf = gpd.GeoDataFrame(
    puntos,
    crs="EPSG:4326",
    geometry=gpd.points_from_xy(
        puntos["_Registra la ubicación del punto_longitude"],
        puntos["_Registra la ubicación del punto_latitude"],
    ),
)

In [ ]:
zona_carabayllo = gpd.read_file("inputs/zona_carabayllo.geojson", crs=4326)
zona_pachacamac = gpd.read_file("inputs/zona_pachacamac.geojson", crs=4326)

In [ ]:
zona_carabayllo.geometry = zona_carabayllo.geometry.apply(make_valid)
zona_pachacamac.geometry = zona_pachacamac.geometry.apply(make_valid)

In [ ]:
zona_carabayllo = gpd.GeoSeries(zona_carabayllo.geometry.unary_union, crs=4326)
zona_pachacamac = gpd.GeoSeries(zona_pachacamac.geometry.unary_union, crs=4326)

In [ ]:
puntos_gdf.intersects(zona_carabayllo.geometry.unary_union).sum()

In [ ]:
puntos_gdf.intersects(zona_pachacamac.geometry.unary_union).sum()

In [ ]:
(
    puntos_gdf.intersects(zona_carabayllo.geometry.unary_union).sum()
    + puntos_gdf.intersects(zona_pachacamac.geometry.unary_union).sum()
)

In [ ]:
data.shape

In [ ]:
pachacamac_markets = pd.DataFrame.from_dict(
    {
        0: {"name": "Mercado Municipal", "lat": -12.22898, "lon": -76.86035},
        1: {"name": "Mercado Ocoña", "lat": -12.22691, "lon": -76.85909},
        2: {"name": "Bioferia Casablanca", "lat": -12.205550, "lon": -76.866392},
    },
    orient="index",
)

pachacamac_markets = gpd.GeoDataFrame(
    pachacamac_markets,
    crs="EPSG:4326",
    geometry=gpd.points_from_xy(pachacamac_markets["lon"], pachacamac_markets["lat"]),
)

In [ ]:
## Final Pachacamac figure
pad = 0.001

ax = pachacamac_markets.plot(color="r", marker="*", markersize=250, figsize=(10, 10))
ax.set_title(f"Pachacamac - Mercados de abastos")

zona_pachacamac.plot(facecolor="none", linewidth=5, ax=ax)

minx, miny, maxx, maxy = zona_pachacamac.total_bounds
ax.set_xlim(minx - pad, maxx + pad)
ax.set_ylim(miny - pad, maxy + pad)
ax.set_axis_off()

for x, y, label in zip(
    pachacamac_markets.geometry.x,
    pachacamac_markets.geometry.y,
    pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

cx.add_basemap(ax, crs="EPSG:4326", source=cx.providers.OpenStreetMap.Mapnik)

plt.savefig(
    "outputs/producto_1/maps/zona_pachacamac_mercados_idrc.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
carabayllo_markets = gpd.GeoDataFrame(
    pd.DataFrame(
        [
            ["Qatuna", -11.89040, -77.04176],
            ["Mercado Modelo Tungasuca", -11.89292, -77.04387],
            ["Bioferia Tungasuca", -11.89664, -77.04062],
            ["Nueva Imagen", -11.89694, -77.03814],
            ["Multiservicios Tungasuca", -11.89457, -77.04273],
        ],
        columns=["name", "lat", "lon"],
    )
)
carabayllo_markets["geometry"] = gpd.points_from_xy(carabayllo_markets["lon"], carabayllo_markets["lat"])

In [ ]:
mcdo_informal = gpd.read_file("inputs/mercado_informal_universitaria_carabayllo.geojson")

In [ ]:
puntos_gdf["Tipo de oferta alimentaria"].value_counts()

In [ ]:
carabayllo = gpd.read_file("inputs/carabayllo/")

In [ ]:
pachacamac = gpd.read_file("inputs/pachacamac/")

In [ ]:
carabayllo_puntos = puntos_gdf.intersects(zona_carabayllo.geometry.unary_union.buffer(0.001))
pachacamac_puntos = puntos_gdf.intersects(zona_pachacamac.geometry.unary_union.buffer(0.001))

In [ ]:
puntos_gdf.loc[carabayllo_puntos, "Distrito"] = "Carabayllo"
puntos_gdf.loc[pachacamac_puntos, "Distrito"] = "Pachacamac"

In [ ]:
puntos_gdf.columns

In [ ]:
puntos_gdf["Distrito"].value_counts()

In [ ]:
puntos_gdf["Nombre"] = puntos_gdf["Nombre"].replace({"X": "SN"})

In [ ]:
puntos_gdf.info()

In [ ]:
(puntos_gdf["Nombre"] == "SN").value_counts()

In [ ]:
puntos_gdf["FLAG_Nombre"] = puntos_gdf["Nombre"] == "SN"

In [ ]:
(
    puntos_gdf.groupby("Distrito")["FLAG_Nombre"]
    .value_counts()
    .to_frame()
    .to_csv("outputs/producto_1/tables/nombre_frecuencias.csv")
)

In [ ]:
(
    puntos_gdf.groupby("Distrito")["Tipo de localización"]
    .value_counts()
    .to_frame()
    .to_csv("outputs/producto_1/tables/tipo_localizacion_frecuencias.csv")
)

In [ ]:
# Draw a nested barplot by species and sex
g = sns.catplot(
    data=puntos_gdf,
    kind="count",
    x="Tipo de oferta alimentaria",
    hue="Distrito",
    ci="sd",
    palette="dark",
    alpha=0.6,
    height=6,
)
g.despine(left=True)
g.set_axis_labels("", "Frecuencia")
g.set_xticklabels(rotation=60)
g.legend.set_title("Disrito")
plt.title("Tipo de oferta alimentaria")

# extract the matplotlib axes_subplot objects from the FacetGrid
ax = g.facet_axis(0, 0)

# iterate through the axes containers
for c in ax.containers:
    labels = [f"{v.get_height():.0f}" for v in c]
    ax.bar_label(c, labels=labels, label_type="edge")

plt.savefig(
    "outputs/producto_1/figures/frecuencias_tipo_oferta_barras.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
ax = gpd.GeoSeries((zona_carabayllo.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", figsize=(10, 10)
)

carabayllo_markets.plot(color="r", marker="*", markersize=500, alpha=0.5, ax=ax)

for x, y, label in zip(
    carabayllo_markets.geometry.x,
    carabayllo_markets.geometry.y,
    carabayllo_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

mcdo_informal.plot(color="r", alpha=0.5, ax=ax)
mcdo_informal_point = mcdo_informal.centroid
ax.annotate(
    "Mercado informal",
    xy=(mcdo_informal_point.x, mcdo_informal_point.y),
    xytext=(3, 3),
    textcoords="offset points",
)

puntos_gdf.plot("Tipo de oferta alimentaria", legend=True, legend_kwds={"loc": "best"}, ax=ax)
leg = ax.get_legend()
leg.set_title("Tipo de oferta alimentaria")

minx, miny, maxx, maxy = zona_carabayllo.total_bounds
margins = 0.001
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

plt.savefig("outputs/producto_1/maps/tipo_oferta_carabayllo.png", dpi=300, bbox_inches="tight")

In [ ]:
ax = gpd.GeoSeries((zona_pachacamac.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", figsize=(10, 10)
)

pachacamac_markets.plot(color="r", marker="*", markersize=250, alpha=0.5, ax=ax)
for x, y, label in zip(
    pachacamac_markets.geometry.x,
    pachacamac_markets.geometry.y,
    pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

puntos_gdf.plot("Tipo de oferta alimentaria", legend=True, legend_kwds={"loc": "lower left"}, ax=ax)
leg = ax.get_legend()
leg.set_title("Tipo de oferta alimentaria")

minx, miny, maxx, maxy = zona_pachacamac.total_bounds
margins = 0.001
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

plt.savefig("outputs/producto_1/maps/tipo_oferta_pachacamac.png", dpi=300, bbox_inches="tight")

In [ ]:
zona_carabayllo_inicial = gpd.read_file("inputs/zona_carabayllo_inicial.geojson", crs=4326)
zona_pachacamac_inicial = gpd.read_file("inputs/zona_pachacamac_inicial.geojson", crs=4326)

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(15, 7.5))

fig.suptitle("Figura 1. Zonas iniciales - Pre Sondeo Rápido")

ax.set_title("Figura 1a. Pachacamac")

gpd.GeoSeries((zona_pachacamac_inicial.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", ax=ax
)

pachacamac_markets.plot(color="r", marker="*", markersize=250, alpha=0.5, ax=ax)
for x, y, label in zip(
    pachacamac_markets.geometry.x,
    pachacamac_markets.geometry.y,
    pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

minx, miny, maxx, maxy = zona_pachacamac_inicial.total_bounds
margins = 0.001
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

ax1.set_title("Figura 1b. Carabayllo")

gpd.GeoSeries((zona_carabayllo_inicial.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", ax=ax1
)

carabayllo_markets.plot(color="r", marker="*", markersize=500, alpha=0.5, ax=ax1)

for x, y, label in zip(
    carabayllo_markets.geometry.x,
    carabayllo_markets.geometry.y,
    carabayllo_markets.name,
):
    ax1.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

mcdo_informal.plot(color="r", alpha=0.5, ax=ax1)
mcdo_informal_point = mcdo_informal.centroid
ax1.annotate(
    "Mercado informal",
    xy=(mcdo_informal_point.x, mcdo_informal_point.y),
    xytext=(3, 3),
    textcoords="offset points",
)

minx, miny, maxx, maxy = zona_carabayllo.total_bounds
margins = 0.001
ax1.set_xlim(minx - margins, maxx + margins)
ax1.set_ylim(miny - margins, maxy + margins)

ax1.set_axis_off()
cx.add_basemap(ax=ax1, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

plt.tight_layout()
plt.savefig("outputs/producto_1/maps/zonas_iniciales.png", dpi=300, bbox_inches="tight")

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(15, 7.5))

fig.suptitle("Figura 2. Zonas de análisis - Post Sondeo Rápido")

ax.set_title("Figura 2a. Pachacamac")

gpd.GeoSeries((zona_pachacamac_inicial.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", marker="--", ax=ax
)

gpd.GeoSeries((zona_pachacamac.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="r", ax=ax
)

pachacamac_markets.plot(color="r", marker="*", markersize=250, alpha=0.5, ax=ax)
for x, y, label in zip(
    pachacamac_markets.geometry.x,
    pachacamac_markets.geometry.y,
    pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

minx, miny, maxx, maxy = zona_pachacamac.total_bounds
margins = 0.001
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

ax1.set_title("Figura 2b. Carabayllo")

gpd.GeoSeries((zona_carabayllo_inicial.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", marker="--", ax=ax1
)

gpd.GeoSeries((zona_carabayllo.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="r", ax=ax1
)

carabayllo_markets.plot(color="r", marker="*", markersize=500, alpha=0.5, ax=ax1)

for x, y, label in zip(
    carabayllo_markets.geometry.x,
    carabayllo_markets.geometry.y,
    carabayllo_markets.name,
):
    ax1.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

mcdo_informal.plot(color="r", alpha=0.5, ax=ax1)
mcdo_informal_point = mcdo_informal.centroid
ax1.annotate(
    "Mercado informal",
    xy=(mcdo_informal_point.x, mcdo_informal_point.y),
    xytext=(3, 3),
    textcoords="offset points",
)

minx, miny, maxx, maxy = zona_carabayllo.total_bounds
margins = 0.001
ax1.set_xlim(minx - margins, maxx + margins)
ax1.set_ylim(miny - margins, maxy + margins)

ax1.set_axis_off()
cx.add_basemap(ax=ax1, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

legend_elements = [
    Line2D([0], [0], color="k", lw=1, label="Área inicial"),
    Line2D([0], [0], color="r", lw=1, label="Área ajustada"),
]

ax1.legend(handles=legend_elements, bbox_to_anchor=(-0.07, 1))

plt.tight_layout()
plt.savefig("outputs/producto_1/maps/zonas_finales.png", dpi=300, bbox_inches="tight")

area
----

0.0021496 km2

2149.6 m2

In [ ]:
puntos_gdf.shape

In [ ]:
puntos_heatmap = puntos_gdf[puntos_gdf["Tipo de oferta alimentaria"] != "Restaurantes"]

In [ ]:
carabayllo_hexs = up.geom.gen_hexagons(10, gpd.GeoDataFrame(geometry=zona_carabayllo.buffer(0.001)))
carabayllo_hexs_counts = up.geom.merge_shape_hex(
    carabayllo_hexs,
    puntos_heatmap[puntos_heatmap["Distrito"] == "Carabayllo"],
    agg={"Distrito": "count"},
)

pachacamac_hexs = up.geom.gen_hexagons(10, gpd.GeoDataFrame(geometry=zona_pachacamac.buffer(0.001)))
pachacamac_hexs_counts = up.geom.merge_shape_hex(
    pachacamac_hexs,
    puntos_heatmap[puntos_heatmap["Distrito"] == "Pachacamac"],
    agg={"Distrito": "count"},
)

In [ ]:
carabayllo_hexs_counts["Distrito"] = carabayllo_hexs_counts["Distrito"].fillna(0)
pachacamac_hexs_counts["Distrito"] = pachacamac_hexs_counts["Distrito"].fillna(0)

In [ ]:
ax = gpd.GeoSeries((zona_carabayllo.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", figsize=(10, 10)
)

gplt.kdeplot(
    puntos_heatmap.query("Distrito == 'Carabayllo'"),
    cmap="Reds",
    ax=ax,
    alpha=0.5,
    shade=True,
    fill=False,
    thresh=0.05,  # cbar=True, cbar_ax=cax,
)

carabayllo_markets.plot(color="r", marker="*", markersize=500, ax=ax)

for x, y, label in zip(
    carabayllo_markets.geometry.x,
    carabayllo_markets.geometry.y,
    carabayllo_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

mcdo_informal.plot(facecolor="none", edgecolor="r", ax=ax)
mcdo_informal_point = mcdo_informal.centroid
ax.annotate(
    "Mercado informal",
    xy=(mcdo_informal_point.x, mcdo_informal_point.y),
    xytext=(3, 3),
    textcoords="offset points",
)

minx, miny, maxx, maxy = zona_carabayllo.total_bounds
margins = 0.001
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

plt.tight_layout()
plt.savefig(
    "outputs/producto_1/maps/tipo_oferta_heatmap_carabayllo.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
ax = gpd.GeoSeries((zona_carabayllo.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", figsize=(10, 10)
)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

carabayllo_hexs_counts.plot("Distrito", legend=True, linewidth=0, cmap="Reds", alpha=0.5, ax=ax, cax=cax)

carabayllo_markets.plot(color="r", marker="*", markersize=500, ax=ax)

for x, y, label in zip(
    carabayllo_markets.geometry.x,
    carabayllo_markets.geometry.y,
    carabayllo_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

mcdo_informal.plot(facecolor="none", edgecolor="r", ax=ax)
mcdo_informal_point = mcdo_informal.centroid
ax.annotate(
    "Mercado informal",
    xy=(mcdo_informal_point.x, mcdo_informal_point.y),
    xytext=(3, 3),
    textcoords="offset points",
)

minx, miny, maxx, maxy = zona_carabayllo.total_bounds
margins = 0.001
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

plt.tight_layout()
plt.savefig(
    "outputs/producto_1/maps/tipo_oferta_hexmap_carabayllo.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
ax = gpd.GeoSeries((zona_pachacamac.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", figsize=(10, 10)
)

gplt.kdeplot(
    puntos_heatmap.query("Distrito == 'Pachacamac'"),
    cmap="Reds",
    ax=ax,
    alpha=0.5,
    shade=True,
    fill=False,
    thresh=0.05,  # cbar=True, cbar_ax=cax,
)

pachacamac_markets.plot(color="r", marker="*", markersize=500, ax=ax)

for x, y, label in zip(
    pachacamac_markets.geometry.x,
    pachacamac_markets.geometry.y,
    pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

minx, miny, maxx, maxy = zona_pachacamac.total_bounds
margins = 0.001
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

plt.tight_layout()
plt.savefig(
    "outputs/producto_1/maps/tipo_oferta_heatmap_pachacamac.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
urba_pachacamac = gpd.read_file("inputs/zona_residencial_pachacamac")

In [ ]:
ax = gpd.GeoSeries((zona_pachacamac.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", figsize=(10, 10)
)

gplt.kdeplot(
    puntos_heatmap.query("Distrito == 'Pachacamac'"),
    cmap="Reds",
    ax=ax,
    alpha=0.5,
    shade=True,
    fill=False,
    thresh=0.05,  # cbar=True, cbar_ax=cax,
)

pachacamac_markets.plot(color="r", marker="*", markersize=500, ax=ax)

for x, y, label in zip(
    pachacamac_markets.geometry.x,
    pachacamac_markets.geometry.y,
    pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")


gpd.GeoSeries((urba_pachacamac.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", ax=ax
)

minx, miny, maxx, maxy = pd.concat([zona_pachacamac, urba_pachacamac["geometry"]]).total_bounds
margins = 0.001
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

plt.tight_layout()
plt.savefig(
    "outputs/producto_1/maps/tipo_oferta_heatmap_pachacamac_con_urbanizacion.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
ax = gpd.GeoSeries((urba_pachacamac.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", figsize=(10, 10)
)

pachacamac_markets.plot(color="r", marker="*", markersize=500, ax=ax)

for x, y, label in zip(
    pachacamac_markets.geometry.x,
    pachacamac_markets.geometry.y,
    pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

minx, miny, maxx, maxy = urba_pachacamac.total_bounds
margins = 0.001
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

plt.tight_layout()
plt.savefig(
    "outputs/producto_1/maps/zona_pachacamac_urbanizacion.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
ax = gpd.GeoSeries((zona_pachacamac.geometry.unary_union.buffer(0.0001).exterior)).plot(  # Smooth borders
    facecolor="none", color="k", figsize=(10, 10)
)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

pachacamac_hexs_counts.plot("Distrito", legend=True, linewidth=0, cmap="Reds", alpha=0.5, ax=ax, cax=cax)

pachacamac_markets.plot(color="r", marker="*", markersize=500, ax=ax)

for x, y, label in zip(
    pachacamac_markets.geometry.x,
    pachacamac_markets.geometry.y,
    pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

minx, miny, maxx, maxy = zona_pachacamac.total_bounds
margins = 0.001
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Positron, crs="EPSG:4326")

plt.tight_layout()
plt.savefig(
    "outputs/producto_1/maps/tipo_oferta_hexmap_pachacamac.png",
    dpi=300,
    bbox_inches="tight",
)

# Mapas Finales Fecha: 2020-06-27

## Mapas de limites, sectores y centralidades


### Carabayllo

Centralidades (mercados):
- Qatuna
- Modelo
- Tungasuca 
- Bioferia
- Informal (Av. Universitaria)


In [ ]:
vec_carabayllo_clean = gpd.read_file("inputs/zona_carabayllo_limpia.geojson")

In [ ]:
vec_carabayllo_clean.crs

In [ ]:
vec_carabayllo_clean

In [ ]:
vec_carabayllo_clean.plot(figsize=(10, 10), facecolor="none", edgecolor="k")

In [ ]:
# nueva_imagen_ix = carabayllo_markets[carabayllo_markets["name"] == "Nueva Imagen"].index
# final_carabayllo_markets = carabayllo_markets.drop(nueva_imagen_ix)
final_carabayllo_markets = carabayllo_markets

In [ ]:
final_carabayllo_markets["name"].values

In [ ]:
final_carabayllo_markets["name"] = [
    "Qatuna",
    "Modelo",
    "Bioferia",
    "Nueva Imagen",
    "Tungasuca",
]

In [ ]:
BASEMAP = cx.providers.Esri.WorldGrayCanvas
MCDO_MARKERSIZE = 100
BIOFERIA_MARKERSIZE = 75
BIOFERIA_COLOR = "green"
MCDO_COLOR = "blue"
MCDO_MARKER = "s"
TEXT_OFFSET = 10
MARGIN = 0.001

In [ ]:
from shapely.geometry.point import Point
from matplotlib_scalebar.scalebar import ScaleBar


points = gpd.GeoSeries(
    [Point(-76.9, -12), Point(-77.9, -12)], crs=4326  # Lima Metropolitana centroid
)  # Geographic WGS 84 - degrees
points = points.to_crs(32718)  # Projected WGS 84 - meters

In [ ]:
distance_meters = points[0].distance(points[1])

In [ ]:
ax = gpd.GeoSeries(vec_carabayllo_clean.unary_union).plot(
    facecolor="none",
    edgecolor="k",
    label="Límite Vecindario Alimentario",
    figsize=(10, 10),
)

# Bioferia
final_carabayllo_markets.query("name == 'Bioferia'").plot(
    color=BIOFERIA_COLOR,
    label="Bioferias",
    marker=MCDO_MARKER,
    markersize=BIOFERIA_MARKERSIZE,
    ax=ax,
)
# Markets
final_carabayllo_markets.query("name != 'Bioferia'").plot(
    color=MCDO_COLOR,
    marker=MCDO_MARKER,
    markersize=MCDO_MARKERSIZE,
    label="Mercados",
    ax=ax,
)

for x, y, label in zip(
    final_carabayllo_markets.geometry.x,
    final_carabayllo_markets.geometry.y,
    final_carabayllo_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(TEXT_OFFSET + 3, -5), textcoords="offset points")

mcdo_informal_point = mcdo_informal.centroid
mcdo_informal.plot(color=MCDO_COLOR, ax=ax)
ax.annotate(
    "Mercado informal",
    xy=(mcdo_informal_point.x, mcdo_informal_point.y),
    xytext=(TEXT_OFFSET + 12.5, -15),
    textcoords="offset points",
)

# puntos_gdf.plot('Tipo de oferta alimentaria', legend=True, legend_kwds={'loc': 'best'}, ax=ax)
# leg = ax.get_legend()
# leg.set_title('Tipo de oferta alimentaria')

minx, miny, maxx, maxy = vec_carabayllo_clean.total_bounds
margins = MARGIN
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=BASEMAP, crs="EPSG:4326", attribution="")

plt.legend(loc="lower left")

ax.add_artist(ScaleBar(distance_meters, location="lower right", box_alpha=0))

plt.savefig(
    "outputs/producto_3/maps/vecindario+sectores+centralidades_carabayllo.png",
    dpi=300,
    bbox_inches="tight",
)

###  Vecindario Pachacamac

Centralidades (mercados):
- Ocona
- Municipal


In [ ]:
vec_pachacamac_clean = gpd.read_file("inputs/zona_pachacamac_limpia.geojson")

In [ ]:
final_pachacamac_markets = pachacamac_markets.copy()
final_pachacamac_markets["name"] = ["Municipal", "Ocoña", "Bioferia Casablanca"]

In [ ]:
ax = gpd.GeoSeries(vec_pachacamac_clean.unary_union).plot(
    facecolor="none",
    edgecolor="k",
    label="Límite Vecindario Alimentario",
    figsize=(10, 10),
)

final_pachacamac_markets.plot(
    color=MCDO_COLOR,
    marker=MCDO_MARKER,
    markersize=MCDO_MARKERSIZE,
    label="Mercados",
    ax=ax,
)

for x, y, label in zip(
    final_pachacamac_markets.geometry.x,
    final_pachacamac_markets.geometry.y,
    final_pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(5, 5), textcoords="offset points")

minx, miny, maxx, maxy = vec_pachacamac_clean.total_bounds
margins = MARGIN
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)


ax.set_axis_off()
cx.add_basemap(ax=ax, source=BASEMAP, crs="EPSG:4326", attribution="")

plt.legend(loc="lower left")

ax.add_artist(ScaleBar(distance_meters, location="lower right", box_alpha=0))

plt.savefig(
    "outputs/producto_3/maps/vecindario+sectores+centralidades_pachacamac.png",
    dpi=300,
    bbox_inches="tight",
)

### Urbanizacion Casablanca - Pachacamac

In [ ]:
casablanca_y_vecindario = pd.concat([vec_pachacamac_clean, urba_pachacamac])

In [ ]:
ax = vec_pachacamac_clean.iloc[0:1].plot(
    facecolor="none",
    edgecolor="k",
    label="Límite Vecindario Alimentario",
    figsize=(10, 10),
)

urba_pachacamac.plot(ax=ax, facecolor="none", edgecolor="violet", label="Límite Urb. Casablanca")

# Bioferia
final_pachacamac_markets.iloc[2:].plot(
    color=BIOFERIA_COLOR,
    label="Bioferia",
    marker=MCDO_MARKER,
    markersize=BIOFERIA_MARKERSIZE,
    ax=ax,
    zorder=1,
)
# Mercados
final_pachacamac_markets.iloc[:2].plot(
    color=MCDO_COLOR,
    marker=MCDO_MARKER,
    markersize=MCDO_MARKERSIZE,
    label="Mercados",
    ax=ax,
    zorder=3,
)

for x, y, label in zip(
    final_pachacamac_markets.geometry.x,
    final_pachacamac_markets.geometry.y,
    final_pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(5, 5), textcoords="offset points")

minx, miny, maxx, maxy = casablanca_y_vecindario.total_bounds
margins = MARGIN
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)


ax.set_axis_off()
cx.add_basemap(ax=ax, source=BASEMAP, crs="EPSG:4326", attribution="")

# Add label of the urbanization to the legend
handles, labels = ax.get_legend_handles_labels()
handles.insert(0, Line2D([0], [0], color="violet", lw=2, label="Límite Urb. Casablanca"))
labels.insert(0, "Límite Urb. Casablanca")
plt.legend(handles=handles, labels=labels, loc="lower right")

ax.add_artist(ScaleBar(distance_meters, location="lower left", box_alpha=0))

plt.savefig(
    "outputs/producto_3/maps/vecindario+sectores+centralidades_pachacamac_casablanca.png",
    dpi=300,
    bbox_inches="tight",
)

## Mapas de oferta alimentaria

### Carabayllo

In [ ]:
puntos_gdf["Tipo de oferta alimentaria"].value_counts()

In [ ]:
final_puntos_gdf = puntos_gdf.copy()

final_puntos_gdf["Tipo de oferta alimentaria"] = (
    final_puntos_gdf["Tipo de oferta alimentaria"]
    .replace({"Otro": "Otros", "Mercados en áreas públicas": "Mercados"})
    .str.strip()
)

In [ ]:
final_puntos_gdf["Tipo de oferta alimentaria"].value_counts()

In [ ]:
color_tipo_oferta = {
    "Ambulantes": "#4F00F3",
    "Bodegas": "#F37C48",
    "Mini-mercados": "#AC2900",
    "Otros": "#D800F8",
    "Panaderias": "#98B500",
    "Restaurantes": "#00C9F5",
}

In [ ]:
final_carabayllo_markets

In [ ]:
ax = vec_carabayllo_clean.iloc[0:1].plot(facecolor="none", edgecolor="k", figsize=(10, 10))

for tipo_oferta, color in color_tipo_oferta.items():
    final_puntos_gdf[final_puntos_gdf["Distrito"] == "Carabayllo"][
        final_puntos_gdf["Tipo de oferta alimentaria"] == tipo_oferta
    ].plot(color=color, ax=ax, label=tipo_oferta, zorder=2)

# Bioferia
final_carabayllo_markets.query("name == 'Bioferia'").plot(
    color=BIOFERIA_COLOR, label="Bioferias", marker=MCDO_MARKER, markersize=BIOFERIA_MARKERSIZE, ax=ax
)
# Markets
final_carabayllo_markets.query("name != 'Bioferia'").plot(
    color=MCDO_COLOR,
    marker=MCDO_MARKER,
    markersize=MCDO_MARKERSIZE,
    label="Mercados",
    ax=ax,
)

for x, y, label in zip(
    final_carabayllo_markets.geometry.x,
    final_carabayllo_markets.geometry.y,
    final_carabayllo_markets.name,
):
    ax.annotate(label, fontsize=10, xy=(x, y), xytext=(TEXT_OFFSET + 3, -5), textcoords="offset points")

mcdo_informal_point = mcdo_informal.centroid
mcdo_informal.plot(color=MCDO_COLOR, ax=ax)
ax.annotate(
    "Mercado informal",
    fontsize=10,
    xy=(mcdo_informal_point.x, mcdo_informal_point.y),
    xytext=(TEXT_OFFSET + 12.5, -10),
    textcoords="offset points",
)

minx, miny, maxx, maxy = vec_carabayllo_clean.total_bounds
margins = MARGIN
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=BASEMAP, crs="EPSG:4326", attribution="")

plt.legend(
    title="Oferta Alimentaria",
    title_fontproperties={"weight": "bold"},
    loc="lower left",
)

ax.add_artist(ScaleBar(distance_meters, location="lower right", box_alpha=0))

plt.savefig(
    "outputs/producto_3/maps/oferta_alimentaria+centralidades_carabayllo.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
ax = vec_carabayllo_clean.iloc[0:1].plot(facecolor="none", edgecolor="k", figsize=(10, 10))

gplt.kdeplot(
    puntos_heatmap.query("Distrito == 'Carabayllo'"),
    cmap="Reds",
    ax=ax,
    alpha=0.5,
    shade=True,
    fill=False,
    thresh=0.05,  # cbar=True, cbar_ax=cax,
)

# Bioferia
final_carabayllo_markets.query("name == 'Bioferia'").plot(
    color=BIOFERIA_COLOR, label="Bioferias", marker=MCDO_MARKER, markersize=BIOFERIA_MARKERSIZE, ax=ax
)
# Markets
final_carabayllo_markets.query("name != 'Bioferia'").plot(
    color=MCDO_COLOR,
    marker=MCDO_MARKER,
    markersize=MCDO_MARKERSIZE,
    label="Mercados",
    ax=ax,
)

for x, y, label in zip(
    final_carabayllo_markets.geometry.x,
    final_carabayllo_markets.geometry.y,
    final_carabayllo_markets.name,
):
    ax.annotate(label, fontsize=10, xy=(x, y), xytext=(TEXT_OFFSET + 3, -5), textcoords="offset points")

mcdo_informal_point = mcdo_informal.centroid
mcdo_informal.plot(color=MCDO_COLOR, edgecolor=MCDO_COLOR, ax=ax)
ax.annotate(
    "Mercado informal",
    fontsize=10,
    xy=(mcdo_informal_point.x, mcdo_informal_point.y),
    xytext=(TEXT_OFFSET + 12.5, -15),
    textcoords="offset points",
)

minx, miny, maxx, maxy = vec_carabayllo_clean.total_bounds
margins = MARGIN
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=BASEMAP, crs="EPSG:4326", attribution="")

plt.legend(
    title="Oferta Alimentaria",
    title_fontproperties={"weight": "bold"},
    loc="lower left",
)

ax.add_artist(ScaleBar(distance_meters, location="lower right", box_alpha=0))

plt.savefig(
    "outputs/producto_3/maps/oferta_alimentaria+mapa_de_calor+centralidades_carabayllo.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
ax = vec_pachacamac_clean.iloc[0:1].plot(facecolor="none", edgecolor="k", figsize=(10, 10))

for tipo_oferta, color in color_tipo_oferta.items():
    dff = final_puntos_gdf[
        (final_puntos_gdf["Distrito"] == "Pachacamac") & (final_puntos_gdf["Tipo de oferta alimentaria"] == tipo_oferta)
    ]

    if dff.shape[0] > 0:
        dff.plot(color=color, ax=ax, label=tipo_oferta, zorder=2)

final_pachacamac_markets.plot(
    color=MCDO_COLOR,
    marker=MCDO_MARKER,
    markersize=MCDO_MARKERSIZE,
    label="Mercados",
    ax=ax,
    zorder=1,
)

for x, y, label in zip(
    final_pachacamac_markets.geometry.x,
    final_pachacamac_markets.geometry.y,
    final_pachacamac_markets.name,
):
    ax.annotate(label, fontsize=10, xy=(x, y), xytext=(TEXT_OFFSET, 0), textcoords="offset points")

minx, miny, maxx, maxy = vec_pachacamac_clean.total_bounds
margins = MARGIN
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=BASEMAP, crs="EPSG:4326", attribution="")

plt.legend(
    title="Oferta Alimentaria",
    title_fontproperties={"weight": "bold"},
    loc="lower left",
)

ax.add_artist(ScaleBar(distance_meters, location="lower right", box_alpha=0))

plt.savefig(
    "outputs/producto_3/maps/oferta_alimentaria+centralidades_pachacamac.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
ax = vec_pachacamac_clean.iloc[0:1].plot(facecolor="none", edgecolor="k", figsize=(10, 10))

urba_pachacamac.plot(ax=ax, facecolor="none", edgecolor="violet", label="Límite Urb. Casablanca")

for tipo_oferta, color in color_tipo_oferta.items():
    dff = final_puntos_gdf[
        (final_puntos_gdf["Distrito"] == "Pachacamac") & (final_puntos_gdf["Tipo de oferta alimentaria"] == tipo_oferta)
    ]

    if dff.shape[0] > 0:
        dff.plot(color=color, ax=ax, label=tipo_oferta, zorder=2)

# Bioferia
final_pachacamac_markets.iloc[2:].plot(
    color=BIOFERIA_COLOR, label="Bioferia", marker=MCDO_MARKER, markersize=BIOFERIA_MARKERSIZE, ax=ax, zorder=1
)
# Mercados
final_pachacamac_markets.iloc[:2].plot(
    color=MCDO_COLOR,
    marker=MCDO_MARKER,
    markersize=MCDO_MARKERSIZE,
    label="Mercados",
    ax=ax,
    zorder=3,
)


for x, y, label in zip(
    final_pachacamac_markets.geometry.x,
    final_pachacamac_markets.geometry.y,
    final_pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(5, 5), textcoords="offset points")

minx, miny, maxx, maxy = casablanca_y_vecindario.total_bounds
margins = MARGIN
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=BASEMAP, crs="EPSG:4326", attribution="")

plt.legend(
    title="Oferta Alimentaria",
    title_fontproperties={"weight": "bold"},
    loc="lower right",
)

ax.add_artist(ScaleBar(distance_meters, location="lower left", box_alpha=0))

plt.savefig(
    "outputs/producto_3/maps/oferta_alimentaria+centralidades_pachacamac_casablanca.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
ax = vec_pachacamac_clean.iloc[0:1].plot(facecolor="none", edgecolor="k", figsize=(10, 10))

gplt.kdeplot(
    puntos_heatmap.query("Distrito == 'Pachacamac'"),
    cmap="Reds",
    ax=ax,
    alpha=0.5,
    shade=True,
    fill=False,
    thresh=0.05,  # cbar=True, cbar_ax=cax,
)

final_pachacamac_markets.plot(
    color=MCDO_COLOR,
    marker=MCDO_MARKER,
    markersize=MCDO_MARKERSIZE,
    label="Mercados",
    ax=ax,
    zorder=1,
)

for x, y, label in zip(
    final_pachacamac_markets.geometry.x,
    final_pachacamac_markets.geometry.y,
    final_pachacamac_markets.name,
):
    ax.annotate(label, fontsize=10, xy=(x, y), xytext=(TEXT_OFFSET, 0), textcoords="offset points")

minx, miny, maxx, maxy = vec_pachacamac_clean.total_bounds
margins = MARGIN
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=BASEMAP, crs="EPSG:4326", attribution="")

plt.legend(
    title_fontproperties={"weight": "bold"},
    loc="lower left",
)

ax.add_artist(ScaleBar(distance_meters, location="lower right", box_alpha=0))

plt.savefig(
    "outputs/producto_3/maps/oferta_alimentaria+mapa_de_calor+centralidades_pachacamac.png",
    dpi=300,
    bbox_inches="tight",
)

In [ ]:
ax = vec_pachacamac_clean.iloc[0:1].plot(facecolor="none", edgecolor="k", figsize=(10, 10))

urba_pachacamac.plot(ax=ax, facecolor="none", edgecolor="violet", label="Límite Urb. Casablanca")

gplt.kdeplot(
    puntos_heatmap.query("Distrito == 'Pachacamac'"),
    cmap="Reds",
    ax=ax,
    alpha=0.5,
    shade=True,
    fill=False,
    thresh=0.05,  # cbar=True, cbar_ax=cax,
)

# Bioferia
final_pachacamac_markets.iloc[2:].plot(
    color=BIOFERIA_COLOR, label="Bioferia", marker=MCDO_MARKER, markersize=BIOFERIA_MARKERSIZE, ax=ax, zorder=1
)
# Mercados
final_pachacamac_markets.iloc[:2].plot(
    color=MCDO_COLOR,
    marker=MCDO_MARKER,
    markersize=MCDO_MARKERSIZE,
    label="Mercados",
    ax=ax,
    zorder=3,
)


for x, y, label in zip(
    final_pachacamac_markets.geometry.x,
    final_pachacamac_markets.geometry.y,
    final_pachacamac_markets.name,
):
    ax.annotate(label, xy=(x, y), xytext=(5, 5), textcoords="offset points")

minx, miny, maxx, maxy = casablanca_y_vecindario.total_bounds
margins = MARGIN
ax.set_xlim(minx - margins, maxx + margins)
ax.set_ylim(miny - margins, maxy + margins)

ax.set_axis_off()
cx.add_basemap(ax=ax, source=BASEMAP, crs="EPSG:4326", attribution="")

plt.legend(
    title_fontproperties={"weight": "bold"},
    loc="lower right",
)

ax.add_artist(ScaleBar(distance_meters, location="lower left", box_alpha=0))

plt.savefig(
    "outputs/producto_3/maps/oferta_alimentaria+mapa_de_calor+centralidades_pachacamac_casablanca.png",
    dpi=300,
    bbox_inches="tight",
)